# **INSTALLING THE REQUIRED LIBRARIES**

In [1]:
!nvidia-smi

Tue Apr  2 16:03:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.3 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.26.1
    Uninstalling accelerate-0.26.1:
      Successfully uninstalled accelerate-0.26.1
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 36.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install neattext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 1.3 MB/s eta 0:00:0000:01:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch

if torch.cuda.is_available():
    # Print the current device
    device = torch.device("cuda")
    print("Current Device:", device)

    num_gpus = torch.cuda.device_count()

    for i in range(num_gpus):
        print("Device {}: {}".format(i, torch.cuda.get_device_name(i)))
else:
    print("CUDA is not available. Using CPU.")

Current Device: cuda
Device 0: Tesla T4
Device 1: Tesla T4


# **IMPORTING THE LIBRARIES**

In [9]:
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import neattext.functions as nfx
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from imblearn.under_sampling import RandomUnderSampler

In [10]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [11]:
kfold = KFold(n_splits=5, random_state=18, shuffle=True)
# Configuration
EPOCHS = 15
BATCH_SIZE = 5
MAX_LEN = 280

# **DATA LOADING**

In [12]:
train_data = pd.read_csv('/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Training.tsv', sep="\t")
train_data

,tweet_id,text,label
0,1364778927105933315,"Psych Med Twitter, wondering if you can help. ...",0
1,1130719434526511104,@dmx_biographer @HellaChillAF I wouldn’t fuck ...,0
2,1184221813733253120,Harley is autistic.... she has the mental capa...,1
3,1435958387020341248,I just made my son cry before going to school ...,1
4,1112559025927999488,Yooooooo my dad really just said “I’d rather h...,0
...,...,...,...
7393,1232778727186223108,Just got all kinds of dirty looks for ordering...,1
7394,1174782065532780544,"anyways- if they did cause autism, i’d rather ...",0
7395,1052612940438728704,2/4 trouble at work and I could have lost my j...,0
7396,1243556662964862976,@LawyerChamber $LgiLder22 I have severe asthma...,1


In [13]:
print("The Shape of the Train Data : ",train_data.shape)

The Shape of the Train Data :  (7398, 3)


In [14]:
train_data['label'].value_counts()

label
0    5118
1    2280
Name: count, dtype: int64

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7398 entries, 0 to 7397
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  7398 non-null   int64 
 1   text      7398 non-null   object
 2   label     7398 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 173.5+ KB


In [16]:
valid_data = pd.read_csv('/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Validation.tsv', sep='\t')
valid_data

,tweet_id,text,label
0,1265323726570225669,Mom Finds Secret to Homeschooling her Autistic...,0
1,1254180961622900738,I hope our child has my calmness and NOT Andre...,0
2,800336385399885824,Blind Girl With Autism Mimics Whitney Houston ...,0
3,1107948318150152192,"@amyschumer watching #growing, my 2yo son has ...",1
4,1438320313511473154,@robbystarbuck @AmericanAir would rather a mot...,0
...,...,...,...
384,1479160193808273413,my 7yo is being tested for adhd and struggles ...,0
385,1084086161667252225,@attachedmrsL the first reply on the tweet tho...,0
386,1404612356819374080,There's a mom who recorded her autistic son &a...,0
387,1458807608052174856,@itsadollthxng Lmfao girl !!! I’m upset fr cau...,1


In [17]:
print("The Shape of the Validation Data is :",valid_data.shape)

The Shape of the Validation Data is : (389, 3)


In [18]:
valid_data['label'].value_counts()

label
0    254
1    135
Name: count, dtype: int64

In [19]:
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  389 non-null    int64 
 1   text      389 non-null    object
 2   label     389 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 9.2+ KB


In [20]:
train_data, test_data = train_test_split(train_data, test_size=1000, random_state=42)

In [21]:
print(train_data['text'][:3])

611     Today my nephew was playing tag (he has asthma...
6047    shoutout to you for loving me, even through al...
1163    Ya know what I don’t get? Even IF vaccines cau...
Name: text, dtype: object


In [22]:
print("The shape of the train data is : ", train_data.shape)
print("The shape of the test data is : ", test_data.shape)

The shape of the train data is :  (6398, 3)
The shape of the test data is :  (1000, 3)


# **DATA PREPROCESSING**

We see the following anomolies in the tweets
1) There are emojis used in the tweets. </br>
2) The label count is not equal. </br>
3) Use of Hastags. </br>
4) Use of attherate symbol (Tag other users). </br>
5) Use of contractions (like yo-year old, can't - cannot, won't-wont etc...). </br>

In [23]:
train_data['tweet'] = train_data['text'].apply(nfx.remove_userhandles)
test_data['tweet'] = test_data['text'].apply(nfx.remove_userhandles)
valid_data['tweet'] = valid_data['text'].apply(nfx.remove_userhandles)
print(train_data[:1])
print(test_data[:1])
print(valid_data[:1])

                tweet_id                                               text  \
611  1035360143100272640  Today my nephew was playing tag (he has asthma...   

     label                                              tweet  
611      0  Today my nephew was playing tag (he has asthma...  
                 tweet_id                                               text  \
2473  1265305799632379904  I really don’t wanna see that video of that la...   

      label                                              tweet  
2473      0  I really don’t wanna see that video of that la...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [24]:
train_data['tweet'] = train_data['tweet'].apply(nfx.remove_hashtags)
test_data['tweet'] = test_data['tweet'].apply(nfx.remove_hashtags)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.remove_hashtags)
print(train_data[:1])
print(test_data[:1])
print(valid_data[:1])

                tweet_id                                               text  \
611  1035360143100272640  Today my nephew was playing tag (he has asthma...   

     label                                              tweet  
611      0  Today my nephew was playing tag (he has asthma...  
                 tweet_id                                               text  \
2473  1265305799632379904  I really don’t wanna see that video of that la...   

      label                                              tweet  
2473      0  I really don’t wanna see that video of that la...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [25]:
train_data['tweet'] = train_data['tweet'].apply(nfx.remove_emojis)
test_data['tweet'] = test_data['tweet'].apply(nfx.remove_emojis)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.remove_emojis)
print(train_data[:1])
print(test_data[:1])
print(valid_data[:1])

                tweet_id                                               text  \
611  1035360143100272640  Today my nephew was playing tag (he has asthma...   

     label                                              tweet  
611      0  Today my nephew was playing tag (he has asthma...  
                 tweet_id                                               text  \
2473  1265305799632379904  I really don’t wanna see that video of that la...   

      label                                              tweet  
2473      0  I really don’t wanna see that video of that la...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [26]:
train_data['tweet'] = train_data['tweet'].apply(nfx.fix_contractions)
test_data['tweet'] = test_data['tweet'].apply(nfx.fix_contractions)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.fix_contractions)
print(train_data[:1])
print(test_data[:1])
print(valid_data[:1])

                tweet_id                                               text  \
611  1035360143100272640  Today my nephew was playing tag (he has asthma...   

     label                                              tweet  
611      0  today my nephew was playing tag (he has asthma...  
                 tweet_id                                               text  \
2473  1265305799632379904  I really don’t wanna see that video of that la...   

      label                                              tweet  
2473      0  i really don’t wanna see that video of that la...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  mom finds secret to homeschooling her autistic...  


In [27]:
train_data = train_data.drop(['text'], axis=1)
test_data = test_data.drop(['text'], axis=1)
valid_data = valid_data.drop(['text'], axis=1)
print(train_data[:1])
print(test_data[:1])
print(valid_data[:1])

                tweet_id  label  \
611  1035360143100272640      0   

                                                 tweet  
611  today my nephew was playing tag (he has asthma...  
                 tweet_id  label  \
2473  1265305799632379904      0   

                                                  tweet  
2473  i really don’t wanna see that video of that la...  
              tweet_id  label  \
0  1265323726570225669      0   

                                               tweet  
0  mom finds secret to homeschooling her autistic...  


In [28]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

In [29]:
train_data.head(5)

,tweet_id,label,tweet
0,1035360143100272640,0,today my nephew was playing tag (he has asthma...
1,1298982248029405187,0,"shoutout to you for loving me, even through al..."
2,1091229479320674305,0,ya know what i don’t get? even if vaccines cau...
3,1457010308107706376,1,i used to feel the same way until i have a v...
4,1295207260805672960,0,i work with small kids and these would be aw...


In [30]:
valid_data.head(5)

,tweet_id,label,tweet
0,1265323726570225669,0,mom finds secret to homeschooling her autistic...
1,1254180961622900738,0,i hope our child has my calmness and not andre...
2,800336385399885824,0,blind girl with autism mimics whitney houston ...
3,1107948318150152192,1,"watching my 2yo son has asd, you just made..."
4,1438320313511473154,0,would rather a mother suffocate an asthmat...


In [31]:
test_data.head(5)

,tweet_id,label,tweet
0,1265305799632379904,0,i really don’t wanna see that video of that la...
1,1046187387141935105,0,girl she physically attacked a kid with autism...
2,1042639916113567744,0,i am the hyperlexic bilingual autistic child o...
3,1000446461702688769,0,so i was at geylang and beside me was the gurl...
4,1044665453820473345,0,"my autistic son hates his photo taken, so i le..."


In [32]:
new_train_data = pd.concat([train_data, valid_data], axis=0)
new_train_data.head(5)

,tweet_id,label,tweet
0,1035360143100272640,0,today my nephew was playing tag (he has asthma...
1,1298982248029405187,0,"shoutout to you for loving me, even through al..."
2,1091229479320674305,0,ya know what i don’t get? even if vaccines cau...
3,1457010308107706376,1,i used to feel the same way until i have a v...
4,1295207260805672960,0,i work with small kids and these would be aw...


In [33]:
new_train_data.tail(5)

,tweet_id,label,tweet
384,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
385,1084086161667252225,0,"the first reply on the tweet though 🤣 yes, i..."
386,1404612356819374080,0,there is a mom who recorded her autistic son &...
387,1458807608052174856,1,lmfao girl !!! i’m upset fr cause jacelyn ha...
388,1299153969009184768,1,i have a child that faces more adver...


In [34]:
new_train_data['label'].value_counts()

label
0    4662
1    2125
Name: count, dtype: int64

In [35]:
print("The shape of the new train data is : ",new_train_data.shape)

The shape of the new train data is :  (6787, 3)


In [36]:
new_train_data.reset_index(drop=True, inplace=True)

In [37]:
new_train_data.head(5)

,tweet_id,label,tweet
0,1035360143100272640,0,today my nephew was playing tag (he has asthma...
1,1298982248029405187,0,"shoutout to you for loving me, even through al..."
2,1091229479320674305,0,ya know what i don’t get? even if vaccines cau...
3,1457010308107706376,1,i used to feel the same way until i have a v...
4,1295207260805672960,0,i work with small kids and these would be aw...


In [38]:
new_train_data.tail(5)

,tweet_id,label,tweet
6782,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
6783,1084086161667252225,0,"the first reply on the tweet though 🤣 yes, i..."
6784,1404612356819374080,0,there is a mom who recorded her autistic son &...
6785,1458807608052174856,1,lmfao girl !!! i’m upset fr cause jacelyn ha...
6786,1299153969009184768,1,i have a child that faces more adver...


In [39]:
undersampler = RandomUnderSampler(random_state=42)

X_resampled, Y_resampled = undersampler.fit_resample(new_train_data[['tweet_id','tweet']], new_train_data['label'])
new_train_data = pd.DataFrame(X_resampled, columns=['tweet_id','tweet'])
new_train_data['label'] = Y_resampled

print(new_train_data['label'].value_counts())

label
0    2125
1    2125
Name: count, dtype: int64


In [40]:
new_train_data.reset_index(drop=True, inplace=True)

In [41]:
def get_train_dataset(x_data, y_data, batch_size, shuffle=True):
    dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

def get_valid_dataset(x_data, y_data, batch_size):
    dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

# **TRAIN THE MODEL**

In [42]:
models = ['roberta-large']

In [43]:
preds = []
weights = []      #f1_score_arrays

In [44]:
class CustomModel(nn.Module):
    def __init__(self, model_name, max_len):
        super(CustomModel, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

In [45]:
def calculate_f1_score(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    print("TP's",true_positives)
    print("FP's",false_positives)
    print("FN's",false_negatives)
    print("TN's",true_negatives)
    precision = true_positives / (true_positives + false_positives + 1e-12)
    recall = true_positives / (true_positives + false_negatives + 1e-12)
    
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-12)
    
    return f1_score

In [46]:
def calculate_precision(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    precision = true_positives / (true_positives + false_positives + 1e-12)
    
    return precision

In [47]:
def calculate_recall(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    recall = true_positives / (true_positives + false_negatives + 1e-12)
    
    return recall

In [48]:
from sklearn.metrics import f1_score

def train_model(model, train_loader, valid_loader, optimizer, criterion, scheduler, fold):
    print("Initializing the loop")
    best_f1_score = -np.inf
    for epoch in range(EPOCHS):
        model.train()
        train_losses = [] 
        # Training loop
        for inputs, targets, attention_mask in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(inputs, attention_mask=attention_mask)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # Update model parameters

            train_losses.append(loss.item())

        # Validation loop
        model.eval()
        with torch.no_grad():
            valid_losses = []
            valid_preds = []
            valid_targets = []
            for inputs, targets, attention_mask in valid_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                attention_mask = attention_mask.to(device)

                outputs = model(inputs, attention_mask=attention_mask)
                loss = criterion(outputs, targets)
                valid_losses.append(loss.item())
                valid_preds.extend(torch.argmax(outputs, axis=1).cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())

            valid_preds = np.array(valid_preds)
            print("The no of valid_preds ",len(valid_preds))
            #print(valid_preds)
            valid_targets = np.array(valid_targets)
            print("The no of valid_targets ",len(valid_targets))
            #print(valid_targets)
            valid_f1_score = calculate_f1_score(valid_targets, valid_preds)
            valid_precision_score = calculate_precision(valid_targets, valid_preds)
            valid_recall_score = calculate_recall(valid_targets, valid_preds)

            avg_train_loss = np.mean(train_losses)
            avg_valid_loss = np.mean(valid_losses)

            print(f"Epoch {epoch + 1}/{EPOCHS}  Train Loss: {avg_train_loss:.4f}  Valid Loss: {avg_valid_loss:.4f}  Valid F1 Score: {valid_f1_score} Precision Score: {valid_precision_score}  Recall Score: {valid_recall_score}")

            # Update best model if necessary
            if valid_f1_score > best_f1_score:
                best_f1_score = valid_f1_score
                torch.save(model.state_dict(), f'fold-{fold}.pt')  # Save model state dictionary

    return best_f1_score

In [49]:
for fold, (train_idx, valid_idx) in enumerate(kfold.split(new_train_data.tweet, new_train_data.label)):
    print('#### FOLD', fold+1)
    x_train, x_valid = new_train_data.tweet[train_idx], new_train_data.tweet[valid_idx]
    y_train, y_valid = new_train_data.label[train_idx], new_train_data.label[valid_idx]

    tokenizer = AutoTokenizer.from_pretrained(models[0])

    # Tokenize data
    x_fake_train = tokenizer(x_train.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    x_fake_valid = tokenizer(x_valid.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")

    # Convert labels to tensors
    y_fake_train = torch.tensor(y_train.tolist())
    y_fake_valid = torch.tensor(y_valid.tolist())

    # Create datasets and data loaders
    train_dataset = get_train_dataset(x_fake_train, y_fake_train, batch_size=BATCH_SIZE)
    valid_dataset = get_valid_dataset(x_fake_valid, y_fake_valid, batch_size=BATCH_SIZE)

    model = CustomModel(model_name=models[0], max_len=MAX_LEN)
    model = model.to(device)

    # Find optimal learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=15e-8)  # Set initial learning rate to a very small value

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True)
    best_f1_score = train_model(model, train_dataset, valid_dataset, optimizer, criterion, scheduler, fold)

#### FOLD 1


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Initializing the loop
The no of valid_preds  850
The no of valid_targets  850
TP's 334
FP's 324
FN's 90
TN's 102
Epoch 1/15  Train Loss: 0.6969  Valid Loss: 0.6938  Valid F1 Score: 0.6173752310531265 Precision Score: 0.5075987841945281  Recall Score: 0.7877358490566019
The no of valid_preds  850
The no of valid_targets  850
TP's 373
FP's 346
FN's 51
TN's 80
Epoch 2/15  Train Loss: 0.6965  Valid Loss: 0.6902  Valid F1 Score: 0.652668416447476 Precision Score: 0.5187760778859519  Recall Score: 0.8797169811320733
The no of valid_preds  850
The no of valid_targets  850
TP's 401
FP's 358
FN's 23
TN's 68
Epoch 3/15  Train Loss: 0.6898  Valid Loss: 0.6844  Valid F1 Score: 0.6779374471677553 Precision Score: 0.5283267457180494  Recall Score: 0.9457547169811298
The no of valid_preds  850
The no of valid_targets  850
TP's 315
FP's 207
FN's 109
TN's 219
Epoch 4/15  Train Loss: 0.6744  Valid Loss: 0.6529  Valid F1 Score: 0.6659619450312163 Precision Score: 0.6034482758620677  Recall Score: 0.74292

/tmp/ipykernel_34/3253241000.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

Initializing the loop
The no of valid_preds  850
The no of valid_targets  850
TP's 332
FP's 333
FN's 76
TN's 109
Epoch 1/15  Train Loss: 0.7239  Valid Loss: 0.7031  Valid F1 Score: 0.6188257222735256 Precision Score: 0.4992481203007511  Recall Score: 0.8137254901960764
The no of valid_preds  850
The no of valid_targets  850
TP's 353
FP's 332
FN's 55
TN's 110
Epoch 2/15  Train Loss: 0.7126  Valid Loss: 0.6952  Valid F1 Score: 0.6459286367790369 Precision Score: 0.515328467153284  Recall Score: 0.8651960784313704
The no of valid_preds  850
The no of valid_targets  850
TP's 207
FP's 184
FN's 201
TN's 258
Epoch 3/15  Train Loss: 0.6953  Valid Loss: 0.6820  Valid F1 Score: 0.5181476846052561 Precision Score: 0.5294117647058809  Recall Score: 0.5073529411764693
The no of valid_preds  850
The no of valid_targets  850
TP's 179
FP's 143
FN's 229
TN's 299
Epoch 4/15  Train Loss: 0.6828  Valid Loss: 0.6706  Valid F1 Score: 0.4904109589036152 Precision Score: 0.5559006211180106  Recall Score: 0.43

/tmp/ipykernel_34/3253241000.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

Initializing the loop
The no of valid_preds  850
The no of valid_targets  850
TP's 202
FP's 165
FN's 246
TN's 237
Epoch 1/15  Train Loss: 0.7103  Valid Loss: 0.6910  Valid F1 Score: 0.49570552147189634 Precision Score: 0.5504087193460475  Recall Score: 0.4508928571428561
The no of valid_preds  850
The no of valid_targets  850
TP's 349
FP's 254
FN's 99
TN's 148
Epoch 2/15  Train Loss: 0.7043  Valid Loss: 0.6859  Valid F1 Score: 0.6641294005703944 Precision Score: 0.5787728026533987  Recall Score: 0.7790178571428553
The no of valid_preds  850
The no of valid_targets  850
TP's 177
FP's 132
FN's 271
TN's 270
Epoch 3/15  Train Loss: 0.6968  Valid Loss: 0.6841  Valid F1 Score: 0.4676354029057243 Precision Score: 0.5728155339805806  Recall Score: 0.3950892857142848
The no of valid_preds  850
The no of valid_targets  850
TP's 370
FP's 233
FN's 78
TN's 169
Epoch 4/15  Train Loss: 0.6907  Valid Loss: 0.6690  Valid F1 Score: 0.7040913415789576 Precision Score: 0.6135986733001648  Recall Score: 0.

# **PREDICTION**

In [129]:
new_valid_data = pd.read_csv("/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Validation.tsv", sep="\t")
new_valid_data

,tweet_id,text,label
0,1265323726570225669,Mom Finds Secret to Homeschooling her Autistic...,0
1,1254180961622900738,I hope our child has my calmness and NOT Andre...,0
2,800336385399885824,Blind Girl With Autism Mimics Whitney Houston ...,0
3,1107948318150152192,"@amyschumer watching #growing, my 2yo son has ...",1
4,1438320313511473154,@robbystarbuck @AmericanAir would rather a mot...,0
...,...,...,...
384,1479160193808273413,my 7yo is being tested for adhd and struggles ...,0
385,1084086161667252225,@attachedmrsL the first reply on the tweet tho...,0
386,1404612356819374080,There's a mom who recorded her autistic son &a...,0
387,1458807608052174856,@itsadollthxng Lmfao girl !!! I’m upset fr cau...,1


In [130]:
new_valid_data['tweet'] = new_valid_data['text'].apply(nfx.remove_userhandles)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.remove_hashtags)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.remove_emojis)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.fix_contractions)
new_valid_data = new_valid_data.drop(['text'], axis=1)
new_valid_data.reset_index(drop=True, inplace=True)

In [131]:
new_valid_data

,tweet_id,label,tweet
0,1265323726570225669,0,mom finds secret to homeschooling her autistic...
1,1254180961622900738,0,i hope our child has my calmness and not andre...
2,800336385399885824,0,blind girl with autism mimics whitney houston ...
3,1107948318150152192,1,"watching my 2yo son has asd, you just made..."
4,1438320313511473154,0,would rather a mother suffocate an asthmat...
...,...,...,...
384,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
385,1084086161667252225,0,"the first reply on the tweet though 🤣 yes, i..."
386,1404612356819374080,0,there is a mom who recorded her autistic son &...
387,1458807608052174856,1,lmfao girl !!! i’m upset fr cause jacelyn ha...


In [132]:
tweets = new_valid_data.tweet
labels = new_valid_data.label

In [133]:
tokenized_tweets = tokenizer(tweets.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
encoded_label = torch.tensor(labels.tolist())

In [134]:
valid_pred_dataset = get_valid_dataset(tokenized_tweets, encoded_label, batch_size=BATCH_SIZE)

/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))


In [135]:
model_1 = CustomModel(model_name=models[0], max_len=MAX_LEN)
model_1.load_state_dict(torch.load("/kaggle/working/fold-4.pt"))
model_1 = model_1.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [136]:
def predict(model, dataloader):
    predictions = []
    model.eval()
    with torch.no_grad():
        for input_ids,_,attention_mask in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
    return predictions

In [137]:
predictions = predict(model_1,valid_pred_dataset)

In [85]:
# predictions

In [138]:
f1_score = calculate_f1_score(np.array(encoded_label.tolist()), np.array(predictions))
print("f1_score is : ",f1_score)
precision = calculate_precision(np.array(encoded_label.tolist()), np.array(predictions))
print("precision is : ", precision)
recall = calculate_recall(np.array(encoded_label.tolist()), np.array(predictions))
print(" recall is : ", recall)

TP's 127
FP's 38
FN's 8
TN's 216
f1_score is :  0.846666666666166
precision is :  0.7696969696969651
 recall is :  0.9407407407407338


In [139]:
df = pd.DataFrame()

In [140]:
df = pd.DataFrame({'tweet_id': new_valid_data['tweet_id'], 'label': predictions})
df.to_csv("prediction_task5.tsv", sep="\t", index=False)

In [141]:
del model_1